In [1]:
using Pkg
Pkg.activate("D://ATIS3//Project.toml") # change path 
Pkg.instantiate()

 Activating environment at `D:\ATIS3\Project.toml`


In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278


In [3]:
#read scenario data
scenarios = CSV.read("scenarios_multinomial.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios)[1:length(names(scenarios))-1] # vector of hexagons (strings)
hours = 4 # duration for charging
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

probability = transpose(scenarios.probs) # probability for each scenario

P = [3, 7, 11] # different charger types in kW
charger_cost = [1000, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


LoadError: ArgumentError: "scenarios_multinomial.csv" is not a valid file

In [4]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P,
        charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:size(hexagons,1)) # Set with hexagons
    Σ = Array{Int}(1:size(P,1)) # Set with charger types 
    Ω = Array{Int}(1:size(probability,1)) # Set with scenarios

    # Initialize model    
    m = Model(CPLEX.Optimizer)
    
    # Consider CVaR
    @variable(m, s[1:Ω] >= 0) # auxiliary variable for VaR, indicator for which scenarios may excess VaR.
    @variable(m, η) # the value-at-risk, VaR, is equal to the largest value η ensuring that the probability...
                    # of obtaining a profit less than η is lower than 1 − α.
    α = 0.05
    
    # Define variables
    @variable(m, 0 <= x_charger[i=Γ, j=Σ], Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ, j=Σ, k=Ω]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ, k=Ω]) # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ, j=Σ, k=Ω], Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ, j=Σ, k=Ω], Int)

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(E_supplied[i,j,k] for j in Σ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= x_charger[i,j].*P[j].*hours)

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], E_supplied[i,j,k] <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))


    # ---------------------- CVaR ------------------------
    # Kosten in jedem Szenario
    @expression(m, cost[k=Ω], invest_cost + opportunity_cost.*sum(E_not_supplied[i,k] for i in Γ))
    
    @constraint(m, cost .- η .<= s)
    @expression(m, CVaR, η + 1/(1-α) * dot(probability, s))
    # -----------------------------------------------------
    
    # Objective function 
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, exp_charging_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability[k])
    @objective(m, Min, (1-λ)*(sum(invest_cost) + sum(exp_charging_cost)) 
                      + λ * CVaR)  
    
     @objective(m, Min, (1-λ)*exp_cost + λ * CVaR)
    
    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied
end


charger_siting (generic function with 1 method)

In [22]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied = charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, probability, P, charger_cost, opportunity_cost)

println(charger_in_hex)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.6785000e+07 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 480 rows and 480 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 781 rows, 781 columns, and 1846 nonzeros.
Reduced MIP has 12 binaries, 485 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.47 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 770 rows and 770 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, u